<a href="https://colab.research.google.com/github/felipefreitas93/mestrado_ppgee/blob/master/xlnet_large_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/felipefreitas93/NLPdatasets
!git clone https://github.com/felipefreitas93/xlnet.git

import pandas as pd

#0.029474180617778827, 0.058948361235557654, 0.11789672247111531, 0.23579344494223062, 0.47158688988446124

dataset_used='MPQA'
FRAC = 0.029474180617778827

DATA_DIR = f'NLPdatasets/{dataset_used}'
train = pd.read_csv(DATA_DIR + '/train.tsv', sep='\t', names=['y','x']).dropna().sample(frac=FRAC)
dataset_len = train.shape[0]/FRAC
train.to_csv(DATA_DIR+ '/train.tsv', sep='\t', index=False, header=False)

NUM_TRAIN_STEPS = 5.12*dataset_len*FRAC/32 #batch size #maybe 4 epochs
WARMUP_STEPS = 0.125*5.12*dataset_len*FRAC/32 #batch size

Cloning into 'NLPdatasets'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 64 (delta 18), reused 60 (delta 17), pack-reused 0
Unpacking objects: 100% (64/64), done.
Cloning into 'xlnet'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 38 (delta 7), reused 33 (delta 7), pack-reused 0
Unpacking objects: 100% (38/38), done.


In [0]:
! pip install sentencepiece

     |████████████████████████████████| 1.0MB 2.8MB/s 


In [0]:
# only needs to be done once
! wget https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
! unzip cased_L-12_H-768_A-12.zip 

--2019-09-15 12:35:38--  https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 2404:6800:4008:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433638019 (414M) [application/zip]
Saving to: ‘cased_L-12_H-768_A-12.zip’

cased_L-12_H-768_A- 100%[===================>] 413.55M  51.5MB/s    in 9.9s    

2019-09-15 12:35:48 (41.8 MB/s) - ‘cased_L-12_H-768_A-12.zip’ saved [433638019/433638019]

Archive:  cased_L-12_H-768_A-12.zip
   creating: xlnet_cased_L-12_H-768_A-12/
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.index  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.data-00000-of-00001  
  inflating: xlnet_cased_L-12_H-768_A-12/spiece.model  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.meta  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_config.j

In [0]:
SCRIPTS_DIR = 'xlnet' #@param {type:"string"}
OUTPUT_DIR = 'proc_data/imdb' #@param {type:"string"}
PRETRAINED_MODEL_DIR = 'xlnet_cased_L-12_H-768_A-12' #@param {type:"string"}
CHECKPOINT_DIR = 'exp/imdb' #@param {type:"string"}

In [0]:
%%time
train_command = f'python xlnet/run_classifier.py \
  --do_train=True \
  --do_eval=False \
  --eval_all_ckpt=False \
  --task_name=imdb \
  --data_dir={DATA_DIR} \
  --output_dir={OUTPUT_DIR} \
  --model_dir={CHECKPOINT_DIR} \
  --uncased=False \
  --spiece_model_file={PRETRAINED_MODEL_DIR}/spiece.model \
  --model_config_path={PRETRAINED_MODEL_DIR}/xlnet_config.json \
  --init_checkpoint={PRETRAINED_MODEL_DIR}/xlnet_model.ckpt \
  --max_seq_length=256 \
  --train_batch_size=16 \
  --eval_batch_size=16 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps={int(NUM_TRAIN_STEPS)} \
  --warmup_steps={int(WARMUP_STEPS)} \
  --save_steps=5000 \
  --iterations=500'

! {train_command}





W0915 12:36:01.183628 140147266262912 deprecation_wrapper.py:119] From xlnet/run_classifier.py:639: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0915 12:36:01.183924 140147266262912 deprecation_wrapper.py:119] From xlnet/run_classifier.py:639: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0915 12:36:01.184127 140147266262912 deprecation_wrapper.py:119] From xlnet/run_classifier.py:663: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0915 12:36:01.184344 140147266262912 deprecation_wrapper.py:119] From xlnet/run_classifier.py:664: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0915 12:36:01.239157 140147266262912 deprecation_wrapper.py:119] From /content/xlnet/model_utils.py:27: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W0915 12:36:01.239412 140147266262912 deprecat

In [0]:
%%time
train_command = "python xlnet/run_classifier.py \
  --do_train=False \
  --do_eval=True \
  --eval_all_ckpt=False \
  --task_name=imdb \
  --data_dir="+DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+CHECKPOINT_DIR+" \
  --uncased=False \
  --spiece_model_file="+PRETRAINED_MODEL_DIR+"/spiece.model \
  --model_config_path="+PRETRAINED_MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+PRETRAINED_MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length=256 \
  --train_batch_size=16 \
  --eval_batch_size=16 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps=5000 \
  --warmup_steps=0 \
  --save_steps=1000 \
  --iterations=500"

! {train_command}





W0915 12:38:55.537642 140411489048448 deprecation_wrapper.py:119] From xlnet/run_classifier.py:639: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0915 12:38:55.537836 140411489048448 deprecation_wrapper.py:119] From xlnet/run_classifier.py:639: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0915 12:38:55.537986 140411489048448 deprecation_wrapper.py:119] From xlnet/run_classifier.py:663: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0915 12:38:55.592357 140411489048448 deprecation_wrapper.py:119] From /content/xlnet/model_utils.py:27: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W0915 12:38:55.592653 140411489048448 deprecation_wrapper.py:119] From /content/xlnet/model_utils.py:36: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Single device mode.
I09